In [ ]:
# загрузка датасета

%mkdir dataset
%cd dataset
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip     

In [17]:
pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import pandas as pd
import numpy as np
import corus

In [19]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='1180',
    text='\r\nЭкс-мэр Волгограда возглавил список ЕР на выборах в гордуму\r\n\r\n\r\nпри этом второе место занял первый вице-премьер правительства региона Роланд Херианов, ранее также упрвлявший городом.\r\nПервый зампредседателя правительства Волгоградской области Роман Гребенников, отстраненный от должности мэра Волгограда в 2011 году, возглавил список кандидатов "Единой России" на выборах в городскую думу, сообщил РИА Новости представитель регионального отделения партии.\r\n\r\nВыборы в гордуму Волгограда пройдут в сентябре этого года по смешанной системе. Всего предстоит выбрать 48 депутатов, из которых 24 будут избраны по партийным спискам и столько же по одномандатным округам. Согласно действующему уставу города, депутатам затем предстоит выбрать из своего состава нового главу Волгограда.\r\n\r\n"Общегородской список кандидатов и список по одномандатным округам утверждены на 24-й конференции волгоградского регионального отделения партии. Общегородской сп

In [20]:
pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from razdel import tokenize

In [22]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [23]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [24]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [36]:
from sklearn.metrics import classification_report

In [52]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [53]:
train_x.apply(len).max(axis=0)

55

In [54]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [55]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

Передаем в сетку только сам токен

In [56]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [57]:
len(vectorize_layer.get_vocabulary())

29869

In [58]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [59]:
mmodel = modelNER()

In [60]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [61]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12430/12430 [==============================] - 63s 5ms/step - loss: 0.2965 - accuracy: 0.9138 - val_loss: 0.2101 - val_accuracy: 0.9394
Epoch 2/3
12430/12430 [==============================] - 60s 5ms/step - loss: 0.1262 - accuracy: 0.9624 - val_loss: 0.2904 - val_accuracy: 0.8939
Epoch 3/3
12430/12430 [==============================] - 60s 5ms/step - loss: 0.1100 - accuracy: 0.9655 - val_loss: 0.2285 - val_accuracy: 0.9413


In [62]:
predict_n = mmodel.predict(valid_data)

4144/4144 [==============================] - 9s 2ms/step


In [63]:
class_preds = np.argmax(tf.nn.softmax(predict_n), axis=1)
class_preds = encoder.inverse_transform(class_preds)

In [64]:
valid_y_inv = encoder.inverse_transform(valid_y)

In [65]:
print(classification_report(valid_y_inv, class_preds))

              precision    recall  f1-score   support

    GEOPOLIT       0.91      0.90      0.90      1107
         LOC       0.85      0.78      0.82      1144
       MEDIA       0.89      0.78      0.83       593
         ORG       0.87      0.56      0.68      3402
         OUT       0.94      0.99      0.97     54819
         PER       0.97      0.72      0.82      5229

    accuracy                           0.94     66294
   macro avg       0.91      0.79      0.84     66294
weighted avg       0.94      0.94      0.94     66294



Передаем в сетку токен и соседей

In [66]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [68]:
mmodel = modelNER()

In [69]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [70]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12430/12430 [==============================] - 72s 6ms/step - loss: 0.2941 - accuracy: 0.9146 - val_loss: 0.2091 - val_accuracy: 0.9380
Epoch 2/3
12430/12430 [==============================] - 67s 5ms/step - loss: 0.1254 - accuracy: 0.9626 - val_loss: 0.2466 - val_accuracy: 0.8935
Epoch 3/3
12430/12430 [==============================] - 66s 5ms/step - loss: 0.1105 - accuracy: 0.9655 - val_loss: 0.2502 - val_accuracy: 0.8828


In [71]:
predict_n13 = mmodel.predict(valid_data)

4144/4144 [==============================] - 14s 3ms/step


In [72]:
class_preds = np.argmax(tf.nn.softmax(predict_n13), axis=1)
class_preds = encoder.inverse_transform(class_preds)

In [73]:
print(classification_report(valid_y_inv, class_preds))

              precision    recall  f1-score   support

    GEOPOLIT       0.90      0.90      0.90      1107
         LOC       0.16      0.93      0.28      1144
       MEDIA       0.91      0.78      0.84       593
         ORG       0.84      0.57      0.68      3402
         OUT       0.97      0.92      0.94     54819
         PER       0.97      0.72      0.82      5229

    accuracy                           0.88     66294
   macro avg       0.79      0.80      0.74     66294
weighted avg       0.95      0.88      0.91     66294



Передаем в сетку би-граммы

In [74]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=2,
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [75]:
mmodel = modelNER()

In [76]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [77]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12430/12430 [==============================] - 75s 6ms/step - loss: 0.2950 - accuracy: 0.9139 - val_loss: 0.2099 - val_accuracy: 0.9396
Epoch 2/3
12430/12430 [==============================] - 67s 5ms/step - loss: 0.1249 - accuracy: 0.9628 - val_loss: 0.2179 - val_accuracy: 0.9416
Epoch 3/3
12430/12430 [==============================] - 67s 5ms/step - loss: 0.1095 - accuracy: 0.9657 - val_loss: 0.2220 - val_accuracy: 0.9411


In [78]:
predict_n2 = mmodel.predict(valid_data)

4144/4144 [==============================] - 10s 2ms/step


In [79]:
class_preds = np.argmax(tf.nn.softmax(predict_n2), axis=1)
class_preds = encoder.inverse_transform(class_preds)

In [80]:
print(classification_report(valid_y_inv, class_preds))

              precision    recall  f1-score   support

    GEOPOLIT       0.90      0.90      0.90      1107
         LOC       0.86      0.78      0.82      1144
       MEDIA       0.92      0.78      0.84       593
         ORG       0.85      0.57      0.68      3402
         OUT       0.95      0.99      0.97     54819
         PER       0.97      0.72      0.82      5229

    accuracy                           0.94     66294
   macro avg       0.91      0.79      0.84     66294
weighted avg       0.94      0.94      0.94     66294

